In [1]:
from jina import Flow , Executor , requests , Document , DocumentArray
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Dictionary.csv")
df.head()

,Word,Count,POS,Definition
0,A,1,"""""","""The first letter of the English and of many o..."
1,A,1,"""""","""The name of the sixth tone in the model major..."
2,A,1,"""""","""An adjective commonly called the indefinite ..."
3,A,1,"""""","""In each; to or for each; as """"""""twenty leagu..."
4,A,1,"""prep.""","""In; on; at; by."""


In [3]:
df = df.drop_duplicates().dropna()
df.shape

(175907, 4)

In [4]:
Document(text = 'something')
docs = DocumentArray()
for ind in range(df.shape[0]):
    word = df.iloc[ind,0]
    defn = df.iloc[ind,-1]
    doc = Document(text= word)
    doc.tags['Definition'] = defn
    docs.append(doc)
docs

<jina.types.arrays.document.DocumentArray length=175907 at 1609778892512>

In [5]:
docs[0].json()

'{\n  "id": "9fd6c2ee-398a-11ec-b438-a86daa5deec8",\n  "mime_type": "text/plain",\n  "tags": {\n    "Definition": "\\"The first letter of the English and of many other alphabets. The capital A of the alphabets of Middle and Western Europe  as also the small letter (a) besides the forms in Italic black letter etc. are all descended from the old Latin A which was borrowed from the Greek Alpha of the same form; and this was made from the first letter (/) of the Phoenician alphabet the equivalent of the Hebrew Aleph and itself from the Egyptian origin. The Aleph was a consonant letter with a guttural breath sound that was not an element of Greek articulation; and the Greeks took it to represent their vowel Alpha with the a sound the Phoenician alphabet having no vowel symbols.\\""\n  },\n  "text": "A"\n}'

In [6]:
docs[0].text

'A'

In [7]:
class Clean(Executor):
    @requests
    def clean(self, docs, parameters, **kwargs):
        print(parameters)
        return DocumentArray(list(filter(lambda x : len(x.text) > 0 , docs)))

In [8]:
model = "sentence-transformers/paraphrase-distilroberta-base-v1"

flow = (
    Flow()
    .add(
        name = 'clean_text',
        uses = Clean
    )
    #.add(
    #    name="error_text_encoder",
     #   uses="jinahub://TransformerTorchEncoder",
    #)
    #.add(
    #    name="error_text_indexer",
    #    uses="jinahub://SimpleIndexer"
    #)
    
)

In [9]:
docs = docs[:100]
docs

<jina.types.arrays.document.DocumentArray length=100 at 1609778707760>

In [10]:
#!del -rf workspace

with flow:
    flow.index(
        inputs=docs,
         docs=docs,
         parameters = {'word' : 'Description'}
    )

⠧ 0/2 waiting clean_text gateway to be ready...        gateway@10608[W]:<class 'jina.peapods.runtimes.gateway.grpc.GRPCRuntime'> timeout after waiting for 600000ms, if your executor takes time to load, you may increase --timeout-ready
        gateway@10608[W]:Pea is being closed before being ready. Most likely some other Pea in the Flow or Pod failed to start
⠹ 0/2 waiting clean_text gateway to be ready...     clean_text@10608[W]:<class 'jina.peapods.runtimes.zmq.zed.ZEDRuntime'> timeout after waiting for 600000ms, if your executor takes time to load, you may increase --timeout-ready
     clean_text@10608[W]:Pea is being closed before being ready. Most likely some other Pea in the Flow or Pod failed to start
⠧ 0/2 waiting clean_text gateway to be ready...        gateway@10608[W]:Terminating process after waiting for readiness signal for graceful shutdown
⠇ 0/2 waiting clean_text gateway to be ready...        gateway@10608[W]:Pea is being closed before being ready. Most likely some othe

RuntimeFailToStart: 

In [ ]:
query = Document(text = input('Query product : '))
with flow:
    response = flow.search(inputs = query, return_results = True)

In [ ]:
matches = response[0].docs[0].matches
matches

In [ ]:
for m in matches:
    print()